In [1]:
import cv2
import glob
import pandas as pd
import urllib
import urllib3
from sklearn.model_selection import train_test_split
import os
import requests
import matplotlib.pyplot as plt

In [2]:
import numpy as np

In [3]:
train_images= glob.glob(r"C:/Users/manoj/OneDrive/Desktop/MANOJ_PYTHONFILES/Deep Learning/Krish Naik/Read_images/chest_xray/test/**/*.jpeg")
len(train_images)

624

In [4]:
folders = os.listdir(r"C:\Users\manoj\OneDrive\Desktop\MANOJ_PYTHONFILES\Deep Learning\Krish Naik\Read_images\chest_xray\test") 

In [5]:
folders

['NORMAL', 'PNEUMONIA']

In [6]:
dic_images={}
for fold in  folders:
    dic_images[fold]=glob.glob(r"C:\Users\manoj\OneDrive\Desktop\MANOJ_PYTHONFILES\Deep Learning\Krish Naik\Read_images\chest_xray\test\{}\*.jpeg".format(fold))

In [7]:
len(dic_images["PNEUMONIA"])

390

In [8]:
dic_labels={}
for i,j in enumerate(folders):
    dic_labels[j] = i

In [34]:
dic_labels["NORMAL"]

0

In [10]:
X,y=[],[]
for i,j in  dic_images.items():
    for img in j:
        g = cv2.imread(img)
        c= cv2.resize(g,(224,224))
        X.append(c)
        y.append(dic_labels[i])

In [49]:
X = np.asarray(X).astype('float32')
y = np.asarray(y).astype('float32').reshape((-1,1))

In [50]:
X_resize =X/255.0


In [13]:
import numpy as np
import cv2

import PIL.Image as Image
import os

import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Flatten,Dense,Conv2D,MaxPool2D
from tensorflow.keras.models import Sequential,Model
from keras_tuner.tuners import RandomSearch 

In [14]:
model = Sequential([
    
    
    
])

In [15]:
X_resize.shape

(624, 224, 224, 3)

In [68]:
import tensorflow as tf

def build_model(hp):
    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = inputs
    for i in range(hp.Int('conv_blocks', 3, 5, default=3)):
        filters = hp.Int('filters_' + str(i), 32, 256, step=32)
        for _ in range(2):
            x = tf.keras.layers.Convolution2D(
            filters, kernel_size=(3, 3), padding='same')(x)
            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.ReLU()(x)
            if hp.Choice('pooling_' + str(i), ['avg', 'max']) == 'max':
                x = tf.keras.layers.MaxPool2D(padding='same')(x)
            else:
                x = tf.keras.layers.AvgPool2D(padding='same')(x)
    x = tf.keras.layers.GlobalAvgPool2D()(x)
    x = tf.keras.layers.Dense(
      hp.Int('hidden_size', 30, 100, step=10, default=50),
      activation='relu')(x)
    x = tf.keras.layers.Dropout(
      hp.Float('dropout', 0, 0.5, step=0.1, default=0.5))(x)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs, outputs)
    model.compile(
    optimizer=tf.keras.optimizers.Adam(
      hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')),
    loss='binary_crossentropy', 
    metrics=['accuracy'])
    return model

In [72]:
import keras_tuner as kt

tuner = kt.Hyperband(
    build_model,
    objective='accuracy',
    max_epochs=6,
    hyperband_iterations=2,
     directory=os.path.normpath('C:/'))

INFO:tensorflow:Reloading Oracle from existing project C:\untitled_project\oracle.json
INFO:tensorflow:Reloading Tuner from C:\untitled_project\tuner0.json


In [73]:
tuner.search(X_resize,y)

Trial 57 Complete [01h 48m 59s]
accuracy: 0.9967948794364929

Best accuracy So Far: 1.0
Total elapsed time: 15h 59m 12s

Search: Running Trial #58

Hyperparameter    |Value             |Best Value So Far 
conv_blocks       |4                 |4                 
filters_0         |224               |64                
pooling_0         |avg               |max               
filters_1         |224               |64                
pooling_1         |avg               |avg               
filters_2         |192               |256               
pooling_2         |max               |max               
hidden_size       |70                |70                
dropout           |0.4               |0.1               
learning_rate     |0.00024782        |0.00072364        
filters_3         |160               |64                
pooling_3         |avg               |max               
filters_4         |32                |224               
pooling_4         |avg               |max             

KeyboardInterrupt: 

Hyperparameter    |Value             |Best Value So Far 
conv_blocks       |4                 |4                 
filters_0         |224               |64                
pooling_0         |avg               |max               
filters_1         |224               |64                
pooling_1         |avg               |avg               
filters_2         |192               |256               
pooling_2         |max               |max               
hidden_size       |70                |70                
dropout           |0.4               |0.1               
learning_rate     |0.00024782        |0.00072364        
filters_3         |160               |64                
pooling_3         |avg               |max               
filters_4         |32                |224               
pooling_4         |avg               |max               
tuner/epochs      |30                |30                
tuner/initial_e...|10                |10                
tuner/bracket     |2                 |3                 
tuner/round       |2                 |3                 
tuner/trial_id    |a69a473d6609d1f...|4101bf2f9da8dcf...

In [74]:
def build_model(hp):  
    model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=(3,3),
        activation='relu',
        input_shape=(224,224,3),
        padding="same"
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=128, step=16),
        kernel_size=(3,3),
        activation='relu',
        padding="same"
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='binary_crossentropy',
              metrics=['accuracy'])

    return model

In [75]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [76]:
tuner_search=RandomSearch(build_model,
                          objective='accuracy',
                          max_trials=3,
#                            directory="output",
                          project_name="X_ray")

In [77]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

In [78]:
tuner_search.search(X_resize,y,epochs=3,callbacks=[stop_early])


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
conv_1_filter     |80                |?                 
conv_1_kernel     |5                 |?                 
conv_2_filter     |64                |?                 
conv_2_kernel     |3                 |?                 
dense_1_units     |48                |?                 
learning_rate     |0.01              |?                 

Epoch 1/3
 6/20 [========>.....................] - ETA: 2:14 - loss: 674.7247 - accuracy: 0.5417

KeyboardInterrupt: 

In [ ]:
# tuner.search(X_resize,y)

In [ ]:
  model=tuner.get_best_models(num_models=1)[0]

In [ ]:
tf.keras.layers.Conv2D()